# Making Pandas DataFrames from API Requests
In this example, we will use the U.S. Geological Survey's API to grab a JSON object of earthquake data and convert it to a `pandas.DataFrame`.

USGS API: https://earthquake.usgs.gov/fdsnws/event/1/

### Get Data from API

In [1]:
import datetime as dt
import pandas as pd
import requests

yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format': 'geojson',
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
}
response = requests.get(api, params=payload)

# let's make sure the request was OK
response.status_code

200

Response of 200 means OK, so we can pull the data out of the result. Since we asked the API for a JSON payload, we can extract it from the response with the `json()` method.

### Isolate the Data from the JSON Response
We need to check the structures of the response data to know where our data is.

In [2]:
earthquake_json = response.json()
earthquake_json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

The USGS API provides information about our request in the `metadata` key. Note that your result will be different, regardless of the date range you chose, because the API includes a timestamp for when the data was pulled:

In [3]:
earthquake_json['metadata']

{'generated': 1667679200000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2022-10-05&endtime=2022-11-04',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.13.6',
 'count': 10082}

Each element in the JSON array `features` is a row of data for our dataframe.

In [4]:
type(earthquake_json['features'])

list

Your data will be different depending on the date you run this.

In [5]:
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 4.7,
  'place': 'Vanuatu',
  'time': 1667519884886,
  'updated': 1667521988040,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/us7000imc1',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=us7000imc1&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'reviewed',
  'tsunami': 0,
  'sig': 340,
  'net': 'us',
  'code': '7000imc1',
  'ids': ',us7000imc1,',
  'sources': ',us,',
  'types': ',origin,phase-data,',
  'nst': 63,
  'dmin': 7.452,
  'rms': 0.5,
  'gap': 44,
  'magType': 'mb',
  'type': 'earthquake',
  'title': 'M 4.7 - Vanuatu'},
 'geometry': {'type': 'Point', 'coordinates': [167.4579, -15.1216, 220.081]},
 'id': 'us7000imc1'}

### Convert to DataFrame
We need to grab the `properties` section out of every entry in the `features` JSON array to create our dataframe.

In [6]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]
df = pd.DataFrame(earthquake_properties_data)
df.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,4.70,Vanuatu,1667519884886,1667521988040,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",us7000imc1,",",us,",",origin,phase-data,",63.0,7.452000,0.50,44.0,mb,earthquake,M 4.7 - Vanuatu
1,0.85,"7km NW of The Geysers, CA",1667519471080,1667520069351,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nc73802781,",",nc,",",nearby-cities,origin,phase-data,scitech-link,",15.0,0.002067,0.02,53.0,md,earthquake,"M 0.9 - 7km NW of The Geysers, CA"
2,2.20,"52 km SSW of Denali National Park, Alaska",1667518699850,1667518830532,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ak022e43wp9d,",",ak,",",origin,phase-data,",NaN,NaN,0.69,NaN,ml,earthquake,"M 2.2 - 52 km SSW of Denali National Park, Alaska"
3,0.84,"7km NW of The Geysers, CA",1667518671120,1667519409291,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nc73802776,",",nc,",",nearby-cities,origin,phase-data,scitech-link,",8.0,0.016350,0.01,79.0,md,earthquake,"M 0.8 - 7km NW of The Geysers, CA"
4,0.53,"2km NW of The Geysers, CA",1667517820690,1667520493397,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nc73802771,",",nc,",",nearby-cities,origin,phase-data,scitech-link,",11.0,0.013460,0.02,95.0,md,earthquake,"M 0.5 - 2km NW of The Geysers, CA"


### (Optional) Write Data to CSV

In [7]:
df.to_csv('earthquakes.csv', index=False)

<hr>
<div>
    <a href="./2-creating_dataframes.ipynb">
        <button style="float: left;">&#8592; Previous Notebook</button>
    </a>
    <a href="./4-inspecting_dataframes.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>